<a href="https://www.kaggle.com/code/alyamcvay/overseas-scools?scriptVersionId=245482772" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This is trying to connect these two databases: One with e the VA Schools and one to show the indexed cost of living to create overseas Va Approved Scools

# Show File names and Create Vaiables

In [28]:
import os
import pandas as pd 
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

print("Connected Datasets (top-level folders in /kaggle/input/):")
for dirname in os.listdir('/kaggle/input/'):
    if os.path.isdir(os.path.join('/kaggle/input/', dirname)):
        print(f"- {dirname}")

# VARIABLES
VApath = "/kaggle/input/va-schools/ComparisonToolData.xlsx"
COLpath = "/kaggle/input/cost-of-living-index-by-country-by-number-2024/Cost_of_Living_Index_by_Country_2024.csv"

print(f"\nCOLpath: {COLpath}")
print(f"VApath (should now be the full file path): {VApath}")

Connected Datasets (top-level folders in /kaggle/input/):
- va-schools
- cost-of-living-index-by-country-by-number-2024

COLpath: /kaggle/input/cost-of-living-index-by-country-by-number-2024/Cost_of_Living_Index_by_Country_2024.csv
VApath (should now be the full file path): /kaggle/input/va-schools/ComparisonToolData.xlsx


## read Cost of living data
\

In [29]:
df_col_data=pd.read_csv('/kaggle/input/cost-of-living-index-by-country-by-number-2024/Cost_of_Living_Index_by_Country_2024.csv')
df_col_data.head(5)

Rank      Country  Cost of Living Index  Rent Index  \
0     1  Switzerland                 101.1        46.5   
1     2      Bahamas                  85.0        36.7   
2     3      Iceland                  83.0        39.2   
3     4    Singapore                  76.7        67.2   
4     5     Barbados                  76.6        19.0   

   Cost of Living Plus Rent Index  Groceries Index  Restaurant Price Index  \
0                            74.9            109.1                    97.0   
1                            61.8             81.6                    83.3   
2                            62.0             88.4                    86.8   
3                            72.1             74.6                    50.4   
4                            48.9             80.8                    69.4   

   Local Purchasing Power Index  
0                         158.7  
1                          54.6  
2                         120.3  
3                         111.1  
4                          43.5

## Read and filter VA Data

In [30]:
# Get filtered data from VA
df_va_data=pd.read_excel(VApath,sheet_name='Comparison_Tool_Data_Full')
df_foreign = df_va_data[df_va_data['country'] != 'USA']
df_foreign = df_foreign[df_foreign['approved']==True]
df_foreign.head(5)



facility code                          institution  \
9820      11000099               AL AKHAWAYN UNIVERSITY   
9837      11000168              UNIVERSITY OF CAPE TOWN   
9839      11000170                  GRIFFITH UNIVERSITY   
9840      11000171                 UNIVERSITY OF VIENNA   
9841      11000174  ROYAL HOLLOWAY UNIVERSITY OF LONDON   

                       city state  zip       country     type  approved  \
9820                 IFRANE   NaN  NaN       MOROCCO  FOREIGN      True   
9837         7701 CAPE TOWN   NaN  NaN  SOUTH AFRICA  FOREIGN      True   
9839        QUEENSLAND 4222   NaN  NaN     AUSTRALIA  FOREIGN      True   
9840            1010 VIENNA   NaN  NaN       AUSTRIA  FOREIGN      True   
9841  EGHAM SURREY TW20 0EX   NaN  NaN       ENGLAND  FOREIGN      True   

      correspondence  flight  ...  solely requires coe  \
9820           False   False  ...                  1.0   
9837           False   False  ...                  1.0   
9839           False   False  ...                  1.0   
9840           False   False  ...                  1.0   
9841           False   False  ...                  1.0   

     requires coe and criteria poo status   latitude   longitude  \
9820                       NaN      APRVD  33.527856   -5.105240   
9837                       NaN       SUSP -33.968056   18.487778   
9839                       NaN      APRVD -28.002373  153.414599   
9840                       NaN      APRVD  48.213128   16.360686   
9841                       NaN      APRVD  51.429671   -0.558543   

      employer provider school provider in state tuition information  \
9820              False            True                          NaN   
9837              False            True                          NaN   
9839              False            True                          NaN   
9840              False            True                          NaN   
9841              False            True                          NaN   

     vrrap provider  ownership name  
9820            NaN             NaN  
9837            NaN             NaN  
9839            NaN             NaN  
9840            NaN             NaN  
9841            NaN             NaN  

[5 rows x 105 columns]

# Create final Table

## Make the table and add the schools

In [31]:
df_final =pd.DataFrame(df_foreign[['institution','country']]).copy()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 781 entries, 9820 to 51950
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   institution  781 non-null    object
 1   country      781 non-null    object
dtypes: object(2)
memory usage: 18.3+ KB


## Fix Captalization in Countries

In [32]:
df_final['country']=df_final['country'].str.title()
df_final.head(5)

institution       country
9820               AL AKHAWAYN UNIVERSITY       Morocco
9837              UNIVERSITY OF CAPE TOWN  South Africa
9839                  GRIFFITH UNIVERSITY     Australia
9840                 UNIVERSITY OF VIENNA       Austria
9841  ROYAL HOLLOWAY UNIVERSITY OF LONDON       England

## Add COL

In [33]:
df_finalco=pd.merge(
    df_final,
    df_col_data,
    left_on='country',
    right_on='Country',
    how='left'
)

df_finalco.head(5)

institution       country  Rank       Country  \
0               AL AKHAWAYN UNIVERSITY       Morocco  96.0       Morocco   
1              UNIVERSITY OF CAPE TOWN  South Africa  78.0  South Africa   
2                  GRIFFITH UNIVERSITY     Australia  10.0     Australia   
3                 UNIVERSITY OF VIENNA       Austria  11.0       Austria   
4  ROYAL HOLLOWAY UNIVERSITY OF LONDON       England   NaN           NaN   

   Cost of Living Index  Rent Index  Cost of Living Plus Rent Index  \
0                  29.5         7.4                            18.9   
1                  34.5        11.5                            23.5   
2                  70.2        33.4                            52.5   
3                  65.1        22.5                            44.7   
4                   NaN         NaN                             NaN   

   Groceries Index  Restaurant Price Index  Local Purchasing Power Index  
0             29.8                    20.7                          38.6  
1             29.7                    29.5                         102.8  
2             77.3                    62.5                         127.4  
3             66.4                    59.3                         102.4  
4              NaN                     NaN                           NaN

# Make Table readable

In [35]:
html_table_string = df_finalco.to_html(
    index=False, 
    justify='left',               
    float_format='%.2f'           
)
with open("final_table.html", "w", encoding="utf-8") as f:
    f.write(html_table_string)

#Print Final Table
print(html_table_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: left;">
      <th>institution</th>
      <th>country</th>
      <th>Rank</th>
      <th>Country</th>
      <th>Cost of Living Index</th>
      <th>Rent Index</th>
      <th>Cost of Living Plus Rent Index</th>
      <th>Groceries Index</th>
      <th>Restaurant Price Index</th>
      <th>Local Purchasing Power Index</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>AL AKHAWAYN UNIVERSITY</td>
      <td>Morocco</td>
      <td>96.00</td>
      <td>Morocco</td>
      <td>29.50</td>
      <td>7.40</td>
      <td>18.90</td>
      <td>29.80</td>
      <td>20.70</td>
      <td>38.60</td>
    </tr>
    <tr>
      <td>UNIVERSITY OF CAPE TOWN</td>
      <td>South Africa</td>
      <td>78.00</td>
      <td>South Africa</td>
      <td>34.50</td>
      <td>11.50</td>
      <td>23.50</td>
      <td>29.70</td>
      <td>29.50</td>
      <td>102.80</td>
    </tr>
    <tr>
      <td>GRIFFITH UNIVERSITY</td>
      <td>Australi